## Librerías Necesarias

In [1]:
##### VOZ ######
# Text-to-Speech Conversion
import pyttsx3 as tts
# Reconocimiento de Voz
import speech_recognition

##### CHATBOT ######
# Intenciones y funcionalidad de chatbot
from neuralintents import GenericAssistant
# Manejo general del sistema
import sys
# Traducir texto, resumir texto
from transformers import pipeline
# Manejo de vectores
import numpy as np
# Calcular similaridad
import spacy

#######FUENTES DE INFORMACION Y PROCESAMIENTO#######
# Modulo para extraer noticias de Google News
from gnewsclient import gnewsclient
# Acceder a contenido de internet
import requests
# Acceder al texto dentro del html
from bs4 import BeautifulSoup
# Convertir html a texto
import html2text
# Analizar expresiones regulares
import re

## Asistente de Voz

### Voces Disponibles 

In [36]:
# Inciar objeto text-to-speech
speaker = tts.init()
# Obtener voces disponibles del sistema
voices = speaker.getProperty('voices')
# Obtener voces en español
voices_esp = []
print('\nVoces disponibles en Español:\n')


for voice in voices:
    if str(voice.languages).find('es')>0:
        print("Voice: %s" % voice.name)
        print(" - ID: %s" % voice.id)
        print(" - Languages: %s" % voice.languages)
        print(" - Gender: %s" % voice.gender)
        print(" - Age: %s" % voice.age)
        print("\n")
        voices_esp.append(voice)

# Elegir español (VARIA EN SISTEMA OPERATIVO)
#azar = np.random.randint(len(voices_esp))
azar=-1
eleccion = voices_esp[azar].id
print("Eligiendo idioma:",eleccion)
speaker.setProperty("voice", eleccion)


Voces disponibles en Español:

Voice: Diego
 - ID: com.apple.speech.synthesis.voice.diego
 - Languages: ['es_AR']
 - Gender: VoiceGenderMale
 - Age: 35


Voice: Jorge
 - ID: com.apple.speech.synthesis.voice.jorge
 - Languages: ['es_ES']
 - Gender: VoiceGenderMale
 - Age: 35


Voice: Juan
 - ID: com.apple.speech.synthesis.voice.juan.premium
 - Languages: ['es_MX']
 - Gender: VoiceGenderMale
 - Age: 30


Voice: Monica
 - ID: com.apple.speech.synthesis.voice.monica.premium
 - Languages: ['es_ES']
 - Gender: VoiceGenderFemale
 - Age: 30


Voice: Paulina
 - ID: com.apple.speech.synthesis.voice.paulina.premium
 - Languages: ['es_MX']
 - Gender: VoiceGenderFemale
 - Age: 30


Eligiendo idioma: com.apple.speech.synthesis.voice.paulina.premium


### Pruebas iniciales - Asistente en Acción

In [37]:
# Por si el speaker se queda en loop
if speaker._inLoop:
    speaker.endLoop()

# Asistente hablando
speaker.say("Hola mundo, ¡estoy viva!")
# Reproducir sonido y parar
speaker.runAndWait()

## Reconocimiento de Audio

In [6]:
# Iniciar reconocimiento de voz
recognizer=speech_recognition.Recognizer()

### Pruebas iniciales - Reconocimiento en Acción

In [7]:
import os
import shutil

## Ajustes para Mac M1 

In [8]:
print("PATH inicial:\n")
print(os.environ['PATH'],end='\n\n')
os.environ['PATH']+=':/opt/homebrew/Cellar/flac/1.3.3/bin/'
print('PATH modificado:\n')
print(os.environ['PATH'],end='\n\n')
path = os.getenv('PATH')
print("shutil.which nos brinda la localización: %s" % (shutil.which('flac')))

PATH inicial:

/Users/moury/miniforge3/envs/transformers/bin:/Users/moury/miniforge3/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin

PATH modificado:

/Users/moury/miniforge3/envs/transformers/bin:/Users/moury/miniforge3/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/opt/homebrew/Cellar/flac/1.3.3/bin/

shutil.which nos brinda la localización: /Users/moury/miniforge3/envs/transformers/bin/flac


## Speech Recognition 

In [9]:
# Asistente hablando
speaker.say("Por favor, dime algo y te lo traduzco a texto:")
# Reproducir sonido y parar
speaker.runAndWait()

In [11]:
# Abrir Micrófono
with speech_recognition.Microphone() as mic:
    # Reducir ruido ambiental
    recognizer.adjust_for_ambient_noise(mic,duration=0.5)
    print("Escuchando Audio...",end=' ')
    # Escuchar audio
    audio=recognizer.listen(mic)
    print("Audio Escuchado.",end=' ')
    # Reconocer audio
    print("Traduciendo a Texto...\n")
    texto=recognizer.recognize_google(audio,language="es-CO")

# Visualizar audio en texto
print("Audio reconocido:", texto)

# Asistente hablando
speaker.say(texto)
# Reproducir sonido y parar
speaker.runAndWait()

Escuchando Audio... Audio Escuchado. Traduciendo a Texto...

Audio reconocido: Hola cómo estás


##  Prueba con Audio Grabado

En Mac M1 no hay permisos para abrir el puerto de audio.

Sin embargo, en otros OS debería funcionar el siguiente snippet sin problema:

```
import sounddevice as sd

from scipy.io.wavfile import write

fs = 44100  # Sample rate
seconds = 5  # Duration of recording

myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
sd.wait()  # Wait until recording is finished
write('audio_test.wav', fs, myrecording)  # Save as WAV file
```

In [12]:
audio_grabado = 'audio_test.wav'

In [13]:
#Need to install sounddevice
import sounddevice as sd
from scipy.io import wavfile as wav
from IPython.display import Audio

scipy_sample_rate, scipy_audio = wav.read(audio_grabado)
Audio(scipy_audio, rate=scipy_sample_rate)

In [14]:
with speech_recognition.AudioFile(audio_grabado) as fuente:
    
    recognizer.adjust_for_ambient_noise(fuente,duration=1)
    audio = recognizer.listen(fuente)
    texto = recognizer.recognize_google(audio,language="es-CO")

print("Audio Convertido a Texto es:\n")
print(texto)

if speaker._inLoop:
    speaker.endLoop()
# Asistente hablando
speaker.say(texto)
# Reproducir sonido y parar
speaker.runAndWait()

Audio Convertido a Texto es:

estamos llegando al final de este diplomado Gracias profesor Daniel Gracias profesor campo Elías Gracias Laura gracias sole gracias a Dios los esperamos chicos y chicas en febrero 7 en el diplomado avanzado


## Pruebas De Pregunta-Respuesta

In [14]:
# Flag para parar acción
done = False

while not done:
    try:
        with speech_recognition.Microphone() as mic:
            
            recognizer.adjust_for_ambient_noise(mic,duration=0.5)
            audio=recognizer.listen(mic)
            texto=recognizer.recognize_google(audio,language="es-CO") #item
            #print(1)
            
            # Poner transformer aquí
            
            
            if speaker._inLoop:
                speaker.endLoop()
            
            speaker.say("Dijiste: " + texto)
            speaker.runAndWait()
            # Pasar texto a minúscula
            texto=texto.lower()
            print("Dijiste:",texto)

    # Maquina no entender
    except speech_recognition.UnknownValueError:
        recognizer=speech_recognition.Recognizer()
        
        if speaker._inLoop:
            speaker.endLoop()
        # Pedir info de nuevo
        speaker.say("No entendí lo que dijiste, ¿por favor podrías repetir?")
        speaker.runAndWait()
    # Desactivar acciones con palabra clave
    if texto=='chao':
        if speaker._inLoop:
            speaker.endLoop()
        # Frase de despedida
        speaker.say("Chao pescao. Nos vemos por ahí.")
        speaker.runAndWait()
        done=True

Dijiste: chao


# Mapeo

In [15]:
def hola():
    if speaker._inLoop:
        speaker.endLoop()
    speaker.say("Hola, en que puedo ayudarte?")
    speaker.runAndWait()

In [16]:
def salir():
    if speaker._inLoop:
        speaker.endLoop()
    speaker.say("Chao pescao. Nos vemos por ahí.")
    speaker.runAndWait()
    sys.exit()

speaker.endLoop()

In [30]:
def noticia():
    from gnewsclient import gnewsclient
    
    if speaker._inLoop:
        speaker.endLoop()
    speaker.say("Claro que sí. Encontré estos temas:")
    speaker.runAndWait()
    # declare a NewsClient object 
    client = gnewsclient.NewsClient(language='spanish', location='Colombia', max_results=5)
    temas = client.topics
    print("\nTemas:\n")
    for i in range(len(temas)):
        print('- ',temas[i])
        
    if speaker._inLoop:
        speaker.endLoop()
    speaker.say(', '.join(temas))
    speaker.runAndWait()
    
    if speaker._inLoop:
        speaker.endLoop()
    speaker.say("¿Cuál tema te gustaría escuchar?")
    speaker.runAndWait()

In [31]:
def temas():
    from gnewsclient import gnewsclient
    client = gnewsclient.NewsClient(language='spanish', location='Colombia', topic=message, max_results=5)
    # get news feed
    global noticias 
    noticias = client.get_news()
    for i in range(len(noticias)):
        print(noticias[i]['title'],'\n')
        
        if speaker._inLoop:
            speaker.endLoop()
        speaker.say(noticias[i]['title'])
        speaker.runAndWait()
        print(noticias[i]['link'],'\n')
        
    if speaker._inLoop:
        speaker.endLoop()
    speaker.say("¿Cuál te gustaría escuchar?")
    speaker.runAndWait()

In [32]:
def resumen():
    
    import numpy as np
    from transformers import pipeline
    from sentence_utils import split_into_sentences
    
    if speaker._inLoop:
        speaker.endLoop()
    speaker.say("Okey. Qué perezoso!, dame un momento mientras inicio el transformer")
    speaker.runAndWait()
    
    # ultimo numero es la noticia elegida
    noticia_elegida = int(message.split()[-1])-1
    # Obtener link
    url = noticias[noticia_elegida]['link']
    # Obtener conexión
    res = requests.get(url)
    
    print("Extrayendo Noticia")
    speaker.say("Extrayendo Noticia")
    speaker.runAndWait()
    # Obtener contenido
    html_page = res.content
    # Extraer Noticia
    soup = BeautifulSoup(html_page, 'html.parser')
    # Extraer texto
    text = soup.find_all('p')
    # Preprocesamiento
    texto = [texts for texts in list(text) if len(texts)>2]
    
    print("Leyendo Noticia")
    speaker.say("Leyendo Noticia")
    speaker.runAndWait()
    # Limpiar Noticia
    texto_limpio = html2text.html2text(''.join(str(texto[2:])))
    #print(texto_limpio)
    texto_limpio = re.sub(r'\([^)]*\)',' ',texto_limpio)
    texto_limpio = re.sub(r'\n','  ',texto_limpio)
    texto_limpio = re.sub(r'\*',' ',texto_limpio)
    
    # Traducir Noticia al inglés
    print("Traduciendo Noticia al inglés")
    speaker.say("Traduciendo Noticia al inglés")
    speaker.runAndWait()
    translator = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")
    # Separar noticia en frases (Mayor rapidez)
    frase = split_into_sentences(texto_limpio)
    # traducir frase
    sentence = translator(frase)
    # Unir frases
    todo_texto = [sentences['translation_text'] for sentences in sentence]
    todo_texto = ' '.join(todo_texto)
    
    
    # Resumir Noticia
    print("Resumiendo Noticia")
    speaker.say("Resumiendo Noticia")
    speaker.runAndWait()
    summarizer = pipeline("summarization")
    resumen = summarizer(todo_texto)
    resumen_en = resumen[0]['summary_text']
    
    # Traducir resumen a español
    print("Traduciendo de vuelta al español...") 
    speaker.say("Traduciendo de vuelta al español...")    
    speaker.runAndWait()
    translator_en_es = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
    resumen_es_list = translator_en_es(resumen_en)
    resumen_es = resumen_es_list[0]['translation_text']
    
    # Decir y escribir resumen
    print(resumen_es)
    speaker.say(resumen_es)
    speaker.runAndWait()
    
    '''
    # declare a NewsClient object 
    with open('resumen_es.txt') as f:
        lines = f.readlines()
    print(lines)
    speaker.say(lines)
    speaker.runAndWait()'''

In [33]:
# Mappings pregunta-respuesta
mappings={
    "inicio":hola,
    "salir": salir,
    "noticias":noticia,
    "resumen":resumen,
    "temas":temas
}

In [34]:
assistant=GenericAssistant('palabras.json',intent_methods=mappings)
assistant.train_model()

In [40]:
from IPython.display import clear_output

while True:
    try:
        with speech_recognition.Microphone() as mic:
            recognizer.adjust_for_ambient_noise(mic,duration=0.5)
            print("Escuchando...",end=' ')
            audio=recognizer.listen(mic)
            print("Reconociendo...")
            message=recognizer.recognize_google(audio,language="es-CO")
            message=message.lower()
            print(message)
        assistant.request(message)
        
    except speech_recognition.UnknownValueError:
        print("Error. Reiniciando Recognizer...\n")
        recognizer=speech_recognition.Recognizer()

Escuchando... Reconociendo...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
resume las 3
Extrayendo Noticia
Leyendo Noticia
Traduciendo Noticia al inglés
Resumiendo Noticia
Traduciendo de vuelta al español...
Los ministros de Salud de EE.UU., Canadá, Alemania, Francia, Italia, Japón y el Reino Unido se reunirán para discutir la evolución de la situación. La llegada de la variante Omicron fue considerada "preocupante" el viernes por la Organización Mundial de la Salud. La nueva variante ha empujado a muchos países a cerrar sus fronteras con Sudáfrica, incluyendo Sudáfrica.
Escuchando... Reconociendo...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling p

SystemExit: 

nacion = 5, world=3

## Noticias 

In [101]:
topic='Nacion'

In [102]:
# declare a NewsClient object 
client = gnewsclient.NewsClient(language='spanish', location='Colombia', topic=topic, max_results=5)
print(client.topics)

['Top Stories', 'World', 'Nation', 'Business', 'Technology', 'Entertainment', 'Sports', 'Science', 'Health']


In [103]:
# get news feed
noticias = client.get_news()
noticias

[{'title': 'Covid hoy en Colombia: 43 muertes diarias y 2.196 nuevos contagios - Portafolio.co',
  'link': 'https://news.google.com/__i/rss/rd/articles/CBMidWh0dHBzOi8vd3d3LnBvcnRhZm9saW8uY28vZWNvbm9taWEvY2Fzb3MtY29udGFnaW9zLWNvcm9uYXZpcnVzLWVuLWNvbG9tYmlhLWhveS1kb21pbmdvLTI4LWRlLW5vdmllbWJyZS1kZS0yMDIxLTU1OTA1MtIBeWh0dHBzOi8vd3d3LnBvcnRhZm9saW8uY28vYW1wL2Vjb25vbWlhL2Nhc29zLWNvbnRhZ2lvcy1jb3JvbmF2aXJ1cy1lbi1jb2xvbWJpYS1ob3ktZG9taW5nby0yOC1kZS1ub3ZpZW1icmUtZGUtMjAyMS01NTkwNTI?oc=5',
  'media': None},
 {'title': 'Hubo acuerdo: Coalición de la Esperanza hará consulta e incluirá a Alejandro Gaviria - El Colombiano',
  'link': 'https://news.google.com/__i/rss/rd/articles/CBMiWmh0dHBzOi8vd3d3LmVsY29sb21iaWFuby5jb20vY29sb21iaWEvY29hbGljaW9uLWRlLWxhLWVzcGVyYW56YS1sbGVnby1hLWFjdWVyZG8tRk8xNjA5MjE1M9IBAA?oc=5',
  'media': None},
 {'title': 'Murió niño de 3 años en el campamento Embera en el Parque Nacional de Bogotá - infobae América',
  'link': 'https://news.google.com/__i/rss/rd/articles/CBMig

In [104]:
noticias[4]

{'title': 'Presidente Duque expresa solidaridad a Perú por terremoto y ofrece ayuda - Caracol Radio',
 'link': 'https://news.google.com/__i/rss/rd/articles/CBMiR2h0dHBzOi8vY2FyYWNvbC5jb20uY28vcmFkaW8vMjAyMS8xMS8yOC9uYWNpb25hbC8xNjM4MTE5NTQwXzk3MzQ2My5odG1s0gFLaHR0cHM6Ly9jYXJhY29sLmNvbS5jby9yYWRpby8yMDIxLzExLzI4L25hY2lvbmFsLzE2MzgxMTk1NDBfOTczNDYzLmFtcC5odG1s?oc=5',
 'media': None}

In [26]:
print(noticias[4]['title'])
speaker.say(noticias[4]['title'])
speaker.runAndWait()

Por ley, se regulará en colegios el uso de celular hasta noveno grado - El Espectador


In [143]:
noticia_elegida = 4

In [144]:
url = noticias[noticia_elegida]['link']

res = requests.get(url)
html_page = res.content

soup = BeautifulSoup(html_page, 'html.parser')
text = soup.find_all('p')
#text

In [145]:
texto = [texts for texts in list(text) if len(texts)>2]

In [146]:
texto_limpio = html2text.html2text(''.join(str(texto[2:])))
#print(texto_limpio)
texto_limpio = re.sub(r'\([^)]*\)',' ',texto_limpio)
texto_limpio = re.sub(r'\n','  ',texto_limpio)
texto_limpio = re.sub(r'\*',' ',texto_limpio)
print(texto_limpio)
#speaker.say(texto_limpio)
#speaker.runAndWait()

[    También se sintió en otros del centro como    [Cundinamarc] a  ,  Quindío, Tolima, Risaralda, Meta, Guaviare, Boyacá y Caldas.    ,    Las autoridades colombianas no han informado de víctimas o daños materiales y  siguen haciendo un barrido por el país para tener un   balance detallado   de  posibles afectaciones.    ,    "Expresamos toda nuestra solidaridad al hermano pueblo del Perú por el   sismo  de 7,5   que sacudió al país esta madrugada y que tuvo como epicentro la  región amazónica. Estamos atentos a los reportes sobre afectaciones para  coordinar la ayuda que podamos prestar desde  [  Colombia  ] ", escribió  el jefe de Estado    ,    En   Ecuador   , el   Sistema Nacional de Gestión de Riesgos y Emergencias    informó que edificaciones del sur del país sufrieron daños por el terremoto  que hasta el momento no deja víctimas humanas.    ,    Expresamos toda nuestra solidaridad al hermano pueblo del  [#Perú]   por el sismo de 7.5° que sacudió al país esta madrugada y que tu

In [147]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")

In [148]:
from sentence_utils import split_into_sentences
frase = split_into_sentences(texto_limpio)
frase

['[    También se sintió en otros del centro como    [Cundinamarc] a  ,  Quindío, Tolima, Risaralda, Meta, Guaviare, Boyacá y Caldas.',
 ',    Las autoridades colombianas no han informado de víctimas o daños materiales y  siguen haciendo un barrido por el país para tener un   balance detallado   de  posibles afectaciones.',
 ',    "Expresamos toda nuestra solidaridad al hermano pueblo del Perú por el   sismo  de 7,5   que sacudió al país esta madrugada y que tuvo como epicentro la  región amazónica.',
 'Estamos atentos a los reportes sobre afectaciones para  coordinar la ayuda que podamos prestar desde  [  Colombia  ] ", escribió  el jefe de Estado    ,    En   Ecuador   , el   Sistema Nacional de Gestión de Riesgos y Emergencias    informó que edificaciones del sur del país sufrieron daños por el terremoto  que hasta el momento no deja víctimas humanas.',
 ',    Expresamos toda nuestra solidaridad al hermano pueblo del  [#Perú]   por el sismo de 7.',
 '5° que sacudió al país esta madr

In [149]:
sentence = translator(frase)

In [150]:
sentence

[{'translation_text': '[ It was also felt in others of the center like [Cundinamarc] to , Quindío, Tolima, Risaralda, Meta, Guaviare, Boyacá and Caldas.'},
 {'translation_text': 'The Colombian authorities have not reported any casualties or material damage and continue to sweep the country in order to have a detailed account of possible impacts.'},
 {'translation_text': '"We express all our solidarity to the brotherly people of Peru for the earthquake of 7.5 that shook the country this morning and had the Amazon region as its epicenter.'},
 {'translation_text': 'We are attentive to reports on impacts to coordinate the assistance we can provide from [Colombia] ", wrote the head of state , In Ecuador , the National System of Risk and Emergency Management reported that buildings in the south of the country suffered damage from the earthquake that so far leaves no human victims.'},
 {'translation_text': ', We express all our solidarity to the brotherly people of [#Peru] for the earthquake 

In [151]:
todo_texto = [sentences['translation_text'] for sentences in sentence]
todo_texto = ' '.join(todo_texto)
todo_texto

'[ It was also felt in others of the center like [Cundinamarc] to , Quindío, Tolima, Risaralda, Meta, Guaviare, Boyacá and Caldas. The Colombian authorities have not reported any casualties or material damage and continue to sweep the country in order to have a detailed account of possible impacts. "We express all our solidarity to the brotherly people of Peru for the earthquake of 7.5 that shook the country this morning and had the Amazon region as its epicenter. We are attentive to reports on impacts to coordinate the assistance we can provide from [Colombia] ", wrote the head of state , In Ecuador , the National System of Risk and Emergency Management reported that buildings in the south of the country suffered damage from the earthquake that so far leaves no human victims. , We express all our solidarity to the brotherly people of [#Peru] for the earthquake of 7. 5th that shook the country this morning and had the Amazon region as its epicenter. We are attentive to reports on impac

In [152]:
summarizer = pipeline("summarization")

In [153]:
resumen = summarizer(todo_texto)

In [154]:
resumen_en = resumen[0]['summary_text']
resumen_en

" Colombian authorities have not reported any casualties or material damage from the 7.5 earthquake that shook the country this morning and had the Amazon region as its epicenter . Colombia's head of state expresses solidarity to the brotherly people of Peru for the earthquake . Ecuador's National System of Risk and Emergency Management reported that buildings in the south of the country suffered damage ."

In [155]:
summarizer = pipeline("summarization")

In [156]:
resumen = summarizer(todo_texto)

In [157]:
resumen_en = resumen[0]['summary_text']
resumen_en

" Colombian authorities have not reported any casualties or material damage from the 7.5 earthquake that shook the country this morning and had the Amazon region as its epicenter . Colombia's head of state expresses solidarity to the brotherly people of Peru for the earthquake . Ecuador's National System of Risk and Emergency Management reported that buildings in the south of the country suffered damage ."

In [158]:
translator_en_es = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")

In [159]:
resumen_es_list = translator_en_es(resumen_en)

In [160]:
resumen_es = resumen_es_list[0]['translation_text']
resumen_es

'Las autoridades colombianas no han reportado víctimas ni daños materiales por el terremoto de 7.5 que sacudió al país esta mañana y tuvo a la región amazónica como su epicentro . El jefe de Estado de Colombia expresa su solidaridad con el hermano pueblo de Perú por el terremoto . El Sistema Nacional de Gestión de Riesgos y Emergencias de Ecuador informó que los edificios en el sur del país sufrieron daños .'

In [43]:
speaker.say(resumen_es)
speaker.runAndWait()

In [47]:
noticias_encabezados = [noticia['title'] for noticia in noticias]
noticias_encabezados = [re.sub(' -\s.*\S$','',noticias_encabezado) for noticias_encabezado in noticias_encabezados]
noticias_encabezados

['Se presentó y se firmó el decreto de la Ley del Vigilante',
 'Ojo con estas 13 aplicaciones populares en celulares Android que roban información valiosa',
 'Conozca los mejores trucos de WhatsApp en su nueva actualización',
 'WhatsApp vuelve con sus condiciones a usuarios y habría nueva fecha límite para aceptarlas',
 'Que no lo tumben: las claves para identificar si un negocio de criptomonedas es real o no']

In [ ]:
!python -m spacy download es_core_news_lg

In [72]:
nlp = spacy.load('es_core_news_lg')

In [74]:
texto_comparar = nlp('policia')

[nlp(encabezado).similarity(texto_comparar) for encabezado in noticias_encabezados]

[0.12374630824919926,
 0.3852905161744063,
 0.08033732324755001,
 0.23713419326636698,
 -0.10259521986963996]

In [153]:
try:
    with speech_recognition.Microphone() as mic:
        recognizer.adjust_for_ambient_noise(mic,duration=0.2)
        audio=recognizer.listen(mic)
        texto=recognizer.recognize_google(audio,language="es-CO")#item
        similitud = [nlp(encabezado).similarity(nlp(texto)) for encabezado in noticias_encabezados]
        cual_not = noticias_encabezados[np.argmax(np.array(similitud))]
    
except speech_recognition.UnknownValueError:
    recognizer=speech_recognition.Recognizer()
    speaker.say("No entendi lo que dijiste, por favor podrias repetir?")
    speaker.runAndWait()

print(cual_not)
print(texto)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
No caiga con el nuevo emoji de labios mordidos en WhatsApp: tiene 2 significados opuestos
emoji
